### Association Rule (ARL)

Association rules are normally written like this: {Diapers} -> {Beer} which means that there is a strong relationship between customers that purchased diapers and also purchased beer in the same transaction.  An association rule has two parts: an antecedent (if) and a consequent (then). An antecedent is an item found within the data. A consequent is an item found in combination with the antecedent. In the above example, the {Diaper} is the antecedent and the {Beer} is the consequent.

#### Story of the dataset 

The dataset includes the sales of an online store between 2009 and 2011.

###### Variables

    Invoice — Invoice Number “If this code starts with C, it means that the operation has been cancelled..”
    StockCode — Product code “Unique number for each product.” 
    Description — Product Description
    Quantity — Number of products purchased
    InvoiceDate — Invoice Date
    Price — Unit price(Sterling)
    CustomerID - Unique number for each customer
    Country — Name of the country 

In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 500)
pd.set_option('display.expand_frame_repr', False)


In [2]:
df_ = pd.read_excel(r"/Users/firatsoydinc/Desktop/Miuul/Week - 3 /Ders Sonrasi Notlar ve Odevler /Ders Sonrası Notlar/online_retail_dataset.xlsx", sheet_name="Year 2010-2011")


In [3]:
df = df_.copy()
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [4]:
def check_df(dataframe):
    print("##################### Shape #####################")
    print(dataframe.shape)
    print("##################### Types #####################")
    print(dataframe.dtypes)
    print("##################### NA #####################")
    print(dataframe.isnull().sum())
    print("##################### Quantiles #####################")
    print(dataframe.quantile([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

check_df(df)

##################### Shape #####################
(541910, 8)
##################### Types #####################
Invoice                object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID           float64
Country                object
dtype: object
##################### NA #####################
Invoice             0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
Price               0
Customer ID    135080
Country             0
dtype: int64
##################### Quantiles #####################
                 0.00      0.05      0.50      0.95     0.99     1.00
Quantity    -80995.00      1.00      3.00     29.00    100.0  80995.0
Price       -11062.06      0.42      2.08      9.95     18.0  38970.0
Customer ID  12346.00  12626.00  15152.00  17905.00  18212.0  18287.0


    Description and Customer ID columns has missing values. I choose to drop this observation from dataset.

    Quantity and price variables have outliers, because the difference between 0.99 quantile and max value of variable is huge. So I use boxplot approach to remove the outliers.

    Quantity and Price variables have negative values, there shouldn't be any negative value in these variables. 


In [5]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit
    

In [6]:
def retail_data_prep(dataframe):
    dataframe.dropna(inplace=True)
    dataframe = dataframe[~dataframe["Invoice"].str.contains("C", na=False)]
    dataframe = dataframe[dataframe["Quantity"] > 0]
    dataframe = dataframe[dataframe["Price"] > 0]
    replace_with_thresholds(dataframe, "Quantity")
    replace_with_thresholds(dataframe, "Price")
    return dataframe

df = retail_data_prep(df)

In [7]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6.0,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8.0,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


##### To take the description of the item from stock code.

In [8]:
def check_id(dataframe, stock_code):
    product_name = dataframe[dataframe["StockCode"] == stock_code][["Description"]].values[0].tolist()
    print(product_name)

##### This study is focusing on Germany.

In [9]:
df_gr = df[df['Country'] == "Germany"]

#### Invoice Product Matrix

In [10]:
def create_invoice_product_df(dataframe, id=False):
    if id:
        return dataframe.groupby(['Invoice', "StockCode"])['Quantity'].sum().unstack().fillna(0). \
            applymap(lambda x: 1 if x > 0 else 0)
    else:
        return dataframe.groupby(['Invoice', 'Description'])['Quantity'].sum().unstack().fillna(0). \
            applymap(lambda x: 1 if x > 0 else 0)


In [11]:
gr_inv_pro_df = create_invoice_product_df(df_gr, id=True)
gr_inv_pro_df.head(20)

StockCode  10002  10125  10135  11001  15034  15036  15039  16008  16011  16014  16016  16045  16054  16218  16235  16236  16237  16238  16259  17003  20652  20658  20665  20668  20674  20675  20676  20677  20679  20681  20682  20684  20685  20696  20702  20704  20705  20707  20711  20712  20713  20716  20717  20718  20719  20723  20724  20725  20726  20727  20728  20749  20750  20751  20752  20754  20755  20761  20766  20767  20772  20774  20794  20796  20828  20846  20897  20902  20903  20914  20961  20963  20971  20972  20973  20974  20975  20977  20978  20979  20981  20982  20983  20984  20992  20997  20998  21002  21014  21015  21025  21034  21035  21039  21041  21042  21051  21054  21055  21056  21058  21059  21061  21062  21068  21078  21080  21084  21086  21088  21090  21094  21096  21108  21110  21114  21115  21116  21121  21122  21123  21124  21125  21126  21128  21135  21136  21137  21154  21155  21156  21158  21159  21164  21165  21169  21175  21194  21196  21197  21202  21204  21205  21206  21207  21208  21209  21210  21211  21212  21213  21215  21216  21217  21218  21219  21220  21221  21224  21226  21231  21232  21238  21239  21240  21242  21243  21244  21245  21246  21249  21250  21251  21257  21258  21259  21262  21264  21272  21277  21284  21289  21291  21292  21293  21311  21314  21318  21320  21328  21329  21340  21354  21355  21358  21360  21371  21374  21380  21381  21383  21389  21392  21394  21395  21397  21398  21399  21401  21402  21403  21408  21411  21415  21424  21426  21427  21428  21429  21430  21439  21441  21446  21447  21452  21456  21458  21462  21467  21471  21479  21481  21484  21485  21494  21495  21497  21498  21499  21500  21503  21504  21506  21507  21508  21509  21519  21523  21524  21527  21531  21533  21534  21535  21537  21539  21544  21555  21556  21558  21559  21561  21562  21563  21564  21576  21577  21578  21579  21580  21581  21588  21592  21615  21620  21621  21622  21624  21625  21628  21642  21650  21656  21658  21666  21668  21669  21670  21671  21672  21673  21675  21679  21680  21682  21683  21698  21700  21703  21704  21705  21706  21707  21708  21710  21715  21716  21718  21719  21723  21724  21731  21735  21739  21741  21746  21747  21749  21754  21755  21756  21770  21773  21774  21775  21786  21787  21788  21789  21791  21809  21811  21817  21826  21828  21829  21832  21843  21844  21871  21873  21874  21877  21878  21880  21881  21882  21883  21884  21889  21890  21891  21892  21894  21896  21899  21903  21905  21906  21907  21908  21914  21915  21916  21917  21918  21925  21926  21927  21928  21929  21930  21931  21932  21933  21934  21935  21936  21937  21944  21948  21949  21955  21967  21972  21973  21974  21975  21976  21977  21980  21981  21982  21983  21984  21985  21986  21987  21988  21989  21990  21991  21992  21993  22021  22024  22025  22026  22027  22028  22029  22030  22031  22032  22035  22037  22040  22041  22042  22044  22045  22046  22047  22048  22049  22050  22051  22055  22057  22059  22060  22061  22062  22063  22064  22066  22067  22070  22072  22073  22074  22075  22076  22077  22078  22079  22080  22081  22082  22083  22085  22086  22087  22088  22089  22090  22091  22093  22094  22097  22098  22099  22100  22110  22111  22112  22113  22114  22117  22119  22123  22124  22128  22129  22130  22131  22134  22135  22138  22139  22141  22142  22144  22147  22148  22149  22150  22151  22163  22164  22168  22169  22171  22173  22174  22175  22176  22178  22179  22180  22181  22186  22187  22189  22190  22191  22192  22193  22195  22196  22197  22199  22200  22201  22202  22203  22204  22207  22208  22209  22210  22211  22212  22215  22220  22221  22222  22223  22228  22230  22231  22232  22233  22236  22242  22243  22244  22245  22246  22247  22252  22254  22255  22258  22260  22262  22263  22264  22271  22272  22273  22274  22276  22277  22278  22279  22280  22281  22296  22300  22301  22302  22303  22304  22307  22311  22315  22316  22317  22318 

In [12]:
# Calculating the occurrence probabilities of the items at the same shopping 

frequent_itemsets = apriori(gr_inv_pro_df, min_support=0.01, use_colnames=True)
frequent_itemsets.sort_values("support", ascending=False)


,support,itemsets
538,0.818381,(POST)
189,0.245077,(22326)
1864,0.225383,"(POST, 22326)"
191,0.157549,(22328)
1931,0.150985,"(22328, POST)"
220,0.137856,(22423)
242,0.137856,(22554)
1746,0.131291,"(22328, 22326)"
17,0.126915,(20719)
4457,0.124726,"(22328, POST, 22326)"


In [13]:
# Creating the association rule of the items

rules = association_rules(frequent_itemsets, metric="support", min_threshold=0.01)

sorted_rules = rules.sort_values("lift", ascending=False)

In [14]:
product_id_list = [21987,23235,22747]

for i in product_id_list:
    check_id(df_gr, i)



['PACK OF 6 SKULL PAPER CUPS']
['STORAGE TIN VINTAGE LEAF']
["POPPY'S PLAYHOUSE BATHROOM"]


### Recommend Item Function

This function recommends 5 items based on the item you bought.

In [21]:
def recommend_item ():
    item = input('Associated item ')
    item = int(item)
    a=check_id(df_gr,item)
    print(a)
    recommendation_list1 = []
    for i, product in enumerate(sorted_rules["antecedents"]):
        for j in list(product):
            if j == item:
                recommendation_list1.append(list(sorted_rules.iloc[i]["consequents"])[0])
    recom_list_1 = recommendation_list1[0:5]

    for i in recom_list_1:
        check_id(df_gr, i)

In [22]:
recommend_item()

Associated item 21987
['PACK OF 6 SKULL PAPER CUPS']
None
['SET/6 RED SPOTTY PAPER CUPS']
['PACK OF 6 SKULL PAPER PLATES']
['SET/6 RED SPOTTY PAPER CUPS']
['PACK OF 6 SKULL PAPER PLATES']
['PACK OF 20 SKULL PAPER NAPKINS']
